# PanLex Dataset Preparation

This notebook is dedicated at making language pairs from the PanLex Dataset for word-level traduction 

The number of languages will be dictated by a pre-filtered list of languages obtained (kind of arbitrarilly) during the Universal Dependencies Conllu Dataset exploration

A first dataset exploration of the different versions (CSV, SQL and JSON) shows that the sql one seems to lack a few things available in the other ones

note that as the PanLex dataset defines the languages by 3 characters, the 2 characters codes had to be transformed to 3 and some of the 2 characters ones could not be found in 3 char definition, so there is not the same number of langs in 3char and 2char

In [104]:
import os
import sys
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import csv
import json
import sqlite3
import pickle

%matplotlib inline

In [105]:
PANLEX_BASEDIR= "/home/leo/projects/Datasets/text/PanLex"
PANLEX_CSV_DIR= os.path.join(PANLEX_BASEDIR,"panlex-20200601-csv")
PANLEX_SQLITE= os.path.join(PANLEX_BASEDIR,"panlex_lite/db.sqlite")
PANLEX_TRANSLATIONS = os.path.join(PANLEX_BASEDIR, "translations_tuples.pkl")

In [13]:
langs_2char = ['af', 'be', 'bg', 'ca', 'cs', 'cy', 'da', 'de', 'el', 'en', 'es', 'et', 'eu', 'fi', 'fr', 'fro', 'ga', 'gd', 'gl', 'grc', 'gsw', 'he', 'hr', 'hu', 'hy', 'it', 'la', 'lt', 'lv', 'mt', 'nl', 'no', 'pl', 'pt', 'ro', 'ru', 'sk', 'sl', 'sr', 'sv', 'tr', 'uk']
langs_3char = ('afr', 'bel', 'bul', 'cat', 'ces', 'cym', 'dan', 'deu', 'ell', 'eng', 'spa', 'est', 'eus', 'fin', 'fra', 'gle', 'gla', 'glg', 'heb', 'hrv', 'hun', 'hye', 'ita', 'lat', 'lit', 'lav', 'mlt', 'nld', 'nor', 'pol', 'por', 'ron', 'rus', 'slk', 'slv', 'srp', 'swe', 'tur', 'ukr')

In [57]:
len(langs_2char), len(langs_3char)

(42, 39)

In [10]:
scripts = ('Latn', 'Cyrl', 'Grek', 'Hebr')

In [7]:
conn = sqlite3.connect(PANLEX_SQLITE)
c = conn.cursor()

In [14]:
q_langcodes = f"SELECT DISTINCT * from langvar where lang_code in {langs_3char} AND script_expr_txt in {scripts};"

In [15]:
q_langcodes

"SELECT DISTINCT * from langvar where lang_code in ('afr', 'bel', 'bul', 'cat', 'ces', 'cym', 'dan', 'deu', 'ell', 'eng', 'spa', 'est', 'eus', 'fin', 'fra', 'gle', 'gla', 'glg', 'heb', 'hrv', 'hun', 'hye', 'ita', 'lat', 'lit', 'lav', 'mlt', 'nld', 'nor', 'pol', 'por', 'ron', 'rus', 'slk', 'slv', 'srp', 'swe', 'tur', 'ukr') AND script_expr_txt in ('Latn', 'Cyrl', 'Grek', 'Hebr');"

In [23]:
r_langcodes = list(c.execute(q_langcodes))

In [32]:
lang_ids = tuple([i[0] for i in r_langcodes])

In [33]:
q_expr_red = f"SELECT DISTINCT * from expr WHERE langvar in {lang_ids}"

In [34]:
q_expr_red

'SELECT DISTINCT * from expr WHERE langvar in (7, 66, 93, 101, 106, 147, 150, 157, 184, 187, 194, 204, 211, 236, 238, 239, 271, 280, 283, 304, 382, 391, 442, 512, 577, 579, 611, 620, 648, 649, 666, 676, 691, 738, 755, 1246, 1247, 1248, 1264, 1349, 1350, 1612, 1616, 1790, 1842, 1844, 1845, 1943, 2079, 4009, 4014, 5466, 5474, 5475, 5479, 5485, 5492, 5493, 5575, 5576, 5586, 5589, 5597, 5603, 5620, 5628, 5629, 6646, 6674, 6709, 6779, 6835, 6838, 6867, 6868, 6870, 6871, 6885, 6943, 6944, 6945, 6949, 6950, 7247, 7555, 7643, 8002, 8088, 8480, 8481, 9066, 9067, 9068, 9095, 9096, 9097, 9098, 9099, 9110, 9327, 9329, 9340, 9509, 9513, 9529, 9592, 9593, 9594, 9595, 9596, 9597, 9598, 9599, 9600, 9601, 9602, 9603, 9604, 9605, 9606, 9607, 9608, 9609, 9610, 9611, 9612, 9614, 9618, 9619, 9620, 9621, 9629, 10011, 10027, 10030, 10039, 10040, 10041, 10044, 10123, 10161, 10162, 10163, 10164, 10165, 10166, 10167, 10168, 10169, 10175, 10258, 10259, 10260, 10261, 10262, 10265, 10266, 10267, 10268, 10269, 1027

In [35]:
r_expr_red = list(c.execute(q_expr_red))

In [50]:
q_denot_red = f"SELECT meaning, expr, langvar FROM denotationx WHERE langvar IN {lang_ids};"

In [51]:
q_denot_red

'SELECT meaning, expr, langvar FROM denotationx WHERE langvar IN (7, 66, 93, 101, 106, 147, 150, 157, 184, 187, 194, 204, 211, 236, 238, 239, 271, 280, 283, 304, 382, 391, 442, 512, 577, 579, 611, 620, 648, 649, 666, 676, 691, 738, 755, 1246, 1247, 1248, 1264, 1349, 1350, 1612, 1616, 1790, 1842, 1844, 1845, 1943, 2079, 4009, 4014, 5466, 5474, 5475, 5479, 5485, 5492, 5493, 5575, 5576, 5586, 5589, 5597, 5603, 5620, 5628, 5629, 6646, 6674, 6709, 6779, 6835, 6838, 6867, 6868, 6870, 6871, 6885, 6943, 6944, 6945, 6949, 6950, 7247, 7555, 7643, 8002, 8088, 8480, 8481, 9066, 9067, 9068, 9095, 9096, 9097, 9098, 9099, 9110, 9327, 9329, 9340, 9509, 9513, 9529, 9592, 9593, 9594, 9595, 9596, 9597, 9598, 9599, 9600, 9601, 9602, 9603, 9604, 9605, 9606, 9607, 9608, 9609, 9610, 9611, 9612, 9614, 9618, 9619, 9620, 9621, 9629, 10011, 10027, 10030, 10039, 10040, 10041, 10044, 10123, 10161, 10162, 10163, 10164, 10165, 10166, 10167, 10168, 10169, 10175, 10258, 10259, 10260, 10261, 10262, 10265, 10266, 10267,

In [52]:
r_denot_red = list(c.execute(q_denot_red))

In [80]:
denotation = {}

for k,eid,lid in r_denot_red:
    if k in denotation:
        denotation[k].append((eid, lid))
    else:
        denotation[k] = [(eid, lid)]

In [81]:
len(list(denotation.keys()))

24457260

In [76]:
expr = {k:(lid, txt) for (k,lid,txt) in r_expr_red}

In [82]:
len(r_expr_red), len(list(expr.keys()))

(11045996, 11045996)

In [77]:
langvar = {i[0]:i for i in r_langcodes}

In [83]:
len(r_langcodes), len(list(langvar.keys()))

(320, 320)

In [53]:
len(r_denot_red)

46189480

In [66]:
r_denot_red[:10], r_expr_red[:10], r_langcodes[:10]

([(10531794, 43387, 7),
  (130851, 43389, 7),
  (8944708, 43389, 7),
  (127891, 43390, 7),
  (146518, 43390, 7),
  (32087720, 43390, 7),
  (15545017, 43391, 7),
  (28089614, 43391, 7),
  (8938382, 43392, 7),
  (11545106, 43396, 7)],
 [(1077, 7, 'Ander Balties'),
  (43387, 7, 'Afrikaans'),
  (43389, 7, 'Albanees'),
  (43390, 7, 'Amharies'),
  (43391, 7, 'Arabies'),
  (43392, 7, 'Bantoe-'),
  (43393, 7, 'Baskies'),
  (43394, 7, 'Birmees'),
  (43395, 7, 'Bulgaars'),
  (43396, 7, 'Deens')],
 [(7,
   'afr',
   0,
   'afr-000',
   35676171,
   43387,
   'Afrikaans',
   26528845,
   '001',
   18147719,
   'Latn'),
  (66,
   'bel',
   0,
   'bel-000',
   35676228,
   43616,
   'беларуская',
   26528845,
   '001',
   17807488,
   'Cyrl'),
  (93,
   'bul',
   0,
   'bul-000',
   35676251,
   6,
   'български',
   26528845,
   '001',
   17807488,
   'Cyrl'),
  (101,
   'cat',
   0,
   'cat-000',
   35676031,
   43730,
   'català',
   26528845,
   '001',
   18147719,
   'Latn'),
  (106,
   'ces',


In [74]:
# denotation = pd.DataFrame(r_denot_red)
# del(denotation)

In [71]:
# expressions = pd.DataFrame(r_expr_red)
# langvar = pd.DataFrame(r_langcodes)
# del(expressions)
# del(lanvar)

In [62]:
# import gc
# gc.collect()

3853

In [98]:
%%time

expr_tuples = []
expr_errs = []

for v in denotation.values():
    transl = []
    for item in v:
        eid,lid = item
        try:
            lang = langvar[lid][1]
            ex = expr[eid][1]
            transl.append((lang, ex))
        except:
            expr_errs.append((eid, lid))
    expr_tuples.append(transl)

CPU times: user 58.3 s, sys: 1.46 s, total: 59.7 s
Wall time: 59.7 s


In [99]:
len(expr_errs)

0

In [100]:
len(expr_tuples)

24457260

In [103]:
expr_tuples[1256:1265]

[[('afr', 'druk'), ('eng', 'squeeze')],
 [('afr', 'dryf')],
 [('afr', 'dryf'),
  ('deu', 'anfeuern'),
  ('deu', 'jagen'),
  ('deu', 'vor sich hertreiben')],
 [('afr', 'duim'),
  ('bul', 'палец'),
  ('cat', 'polze'),
  ('cym', 'bawd'),
  ('dan', 'tommeltot'),
  ('deu', 'Daumen'),
  ('eng', 'thumb'),
  ('eus', 'hazlodi'),
  ('fin', 'peukalo'),
  ('fra', 'pouce'),
  ('gle', 'ordóg'),
  ('hun', 'hüvelykujj'),
  ('ita', 'ditone'),
  ('ita', 'pollice'),
  ('lat', 'pollex'),
  ('lit', 'nykštys'),
  ('nld', 'duim'),
  ('pol', 'kciuk'),
  ('por', 'dedo polegar'),
  ('ron', 'deget mare'),
  ('rus', 'большой палец'),
  ('slk', 'palec'),
  ('slv', 'palec'),
  ('spa', 'dedo gordo'),
  ('spa', 'pulgar'),
  ('swe', 'tumme'),
  ('tur', 'başparmak')],
 [('afr', 'duister'), ('deu', 'finster')],
 [('afr', 'duister'), ('deu', 'dunkel'), ('deu', 'finster')],
 [('afr', 'Duitse Demokratiese Republiek'),
  ('bul', 'Германска демократична република'),
  ('cat', 'República Democràtica Alemanya'),
  ('ces', 'Něm

In [91]:
expr_errs[:10]

[(43387, 7),
 (43389, 7),
 (783236, 7),
 (1480661, 66),
 (1480662, 66),
 (1525072, 93),
 (43718, 101),
 (1534525, 106),
 (1044039, 147),
 (1582933, 150)]

In [93]:
expr[43387]

(7, 'Afrikaans')

In [106]:
f = open(PANLEX_TRANSLATIONS, 'wb')

pickle.dump(expr_tuples, f)

In [107]:
f.close()